In [3]:
import pandas as pd
from mftool import Mftool
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

In [4]:
mf = Mftool()

# Get scheme codes
scheme_codes = mf.get_scheme_codes()
scheme_code_list_main = [x for x in scheme_codes.keys()]

# Query scheme codes for mid, small and flexi cap
scheme_code_list = [k for k, v in scheme_codes.items() if 'mid' in v.lower()]
scheme_code_list = scheme_code_list+[k for k, v in scheme_codes.items() if 'flexi' in v.lower()]
scheme_code_list = scheme_code_list+[k for k, v in scheme_codes.items() if 'small' in v.lower()]


# Comment the queries and uncomment this line to get all the schemes
# scheme_code_list = scheme_code_list

In [5]:
len(scheme_code_list)

690

In [109]:
def HistoricalNav(scheme_code_list, start_date, end_date):
    count = 0
    main_df = pd.DataFrame()
    for schemes in scheme_code_list:
        count=count+1
#         print('='*25)
#         print(schemes)
        data = mf.get_scheme_historical_nav_for_dates(schemes, start_date, end_date) # requesting NAV data from the api.
        df = pd.DataFrame(data['data']) 
        df['scheme_code'] = pd.Series([data['scheme_code'] for x in range(len(df.index))]) #adding Pandas Series(scheme_code) as a column in Pandas Dataframe.
        df['scheme_name'] = pd.Series([data['scheme_name'] for x in range(len(df.index))]) #adding Pandas Series(scheme_name) as a column in Pandas Dataframe.
        try:
            df = df.sort_values(by = 'date')
            main_df = main_df.append(df)
#             print("Data Fetched!")
        except:
            print('Data doesnt exist of selected date.')
#         print('='*25)
        print(str((count/len(scheme_code_list))*100)+'%')
        clear_output(wait=True)

    main_df = main_df[['scheme_code', 'scheme_name', 'date', 'nav']] #creating names of dataframe columns 
    main_df.reset_index(drop = True, inplace = True) 
    return main_df


In [110]:
def NAV_Data(start,end): 
    try:
        values_df = HistoricalNav(scheme_code_list = scheme_code_list[0:], start_date= start, end_date= end) #to get the data
        return values_df
    except KeyError:
        start=datetime.strptime(start, '%d-%m-%Y') - timedelta(1)
        return NAV_Data(start.strftime("%d-%m-%Y"),end)


In [111]:
start_date= "01-01-2023"
end_date = "08-04-2023"
values_df = NAV_Data(start_date,end_date)
values_df


,scheme_code,scheme_name,date,nav
0,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,01-02-2023,20.18000
1,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,01-03-2023,20.04000
2,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,02-01-2023,20.92000
3,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,02-02-2023,20.20000
4,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,02-03-2023,19.95000
...,...,...,...,...
38029,144988,Sundaram Emerging Small Cap VII Direct Plan - ...,28-03-2023,21.94490
38030,144988,Sundaram Emerging Small Cap VII Direct Plan - ...,29-03-2023,22.24770
38031,144988,Sundaram Emerging Small Cap VII Direct Plan - ...,30-01-2023,21.98370
38032,144988,Sundaram Emerging Small Cap VII Direct Plan - ...,31-01-2023,22.34730


In [112]:
# add datetime, sort and drop duplicates
values_df['datetime'] = pd.to_datetime(values_df['date'], format='%d-%m-%Y')

In [114]:
values_df

,scheme_code,scheme_name,date,nav,datetime
0,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,01-02-2023,20.18000,2023-02-01
1,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,01-03-2023,20.04000,2023-03-01
2,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,02-01-2023,20.92000,2023-01-02
3,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,02-02-2023,20.20000,2023-02-02
4,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,02-03-2023,19.95000,2023-03-02
...,...,...,...,...,...
38029,144988,Sundaram Emerging Small Cap VII Direct Plan - ...,28-03-2023,21.94490,2023-03-28
38030,144988,Sundaram Emerging Small Cap VII Direct Plan - ...,29-03-2023,22.24770,2023-03-29
38031,144988,Sundaram Emerging Small Cap VII Direct Plan - ...,30-01-2023,21.98370,2023-01-30
38032,144988,Sundaram Emerging Small Cap VII Direct Plan - ...,31-01-2023,22.34730,2023-01-31


In [158]:
# get unique scheme names
schemes = values_df['scheme_name'].explode().unique()

In [159]:
# keep only latest records.. skip this cell if you want to keep all the records
final_df = pd.DataFrame()
for scheme in schemes:
    chunk_df = temp[temp['scheme_name']==scheme]
    chunk_df = chunk_df.sort_values(by='datetime')    
    final_df =final_df.append(chunk_df.iloc[-1], ignore_index=True)

In [160]:
final_df

,scheme_code,scheme_name,date,nav,datetime
0,110598,BANK OF INDIA Large & Mid Cap Equity Fund Reg...,06-04-2023,20.14000,2023-04-06
1,119349,BANK OF INDIA Large & Mid Cap Equity Fund Dire...,06-04-2023,17.24000,2023-04-06
2,119300,BANK OF INDIA Large & Mid Cap Equity Fund Dire...,06-04-2023,32.03000,2023-04-06
3,119350,BANK OF INDIA Large & Mid Cap Equity Fund Dire...,06-04-2023,61.99000,2023-04-06
4,119346,BANK OF INDIA Large & Mid Cap Equity Fund Dire...,06-04-2023,16.98000,2023-04-06
...,...,...,...,...,...
583,144727,Sundaram Emerging Small Cap Series VI Regular ...,06-04-2023,21.19680,2023-04-06
584,144989,Sundaram Emerging Small Cap Series VII Direct ...,06-04-2023,21.46200,2023-04-06
585,144987,Sundaram Emerging Small Cap Series VII Regular...,06-04-2023,22.39990,2023-04-06
586,144986,Sundaram Emerging Small Cap Series VII Regular...,06-04-2023,21.00690,2023-04-06


In [161]:
final_df.drop(['datetime'],axis=1, inplace = True)
final_df.to_csv('MF_NAV_DATA.csv')